# This notebook is going to implement the parser for this lab

# 🛠️ Cơ chế hoạt động: Thuật toán tìm File Gốc (Root File Detection)

Để xác định chính xác đâu là file chính (`main.tex`) trong một rừng các file mã nguồn LaTeX hỗn độn, chúng ta không dùng cách đoán mò. Thuật toán sử dụng phương pháp **"Pháp y kỹ thuật số" (Digital Forensics)** kết hợp với **Phân tích Cây phụ thuộc (Dependency Graph)**.

Quy trình xử lý đi qua 3 màng lọc chính:

### 1. Màng lọc Cấu trúc (The Gatekeeper)
Trước tiên, mọi file `.tex` phải chứng minh nó là một văn bản độc lập, không phải là đoạn code rời rạc.
* **Điều kiện tiên quyết:** File bắt buộc phải chứa lệnh `\documentclass`.
* **Loại bỏ:** Các file Plain TeX cũ, file data, hoặc các file chương hồi (chapter) không có khai báo môi trường.

### 2. Phân tích Cây phụ thuộc (Dependency Graph Analysis)
Đây là bước quan trọng nhất để giải quyết xung đột khi có nhiều file cùng chạy được (ví dụ dùng gói `subfiles`).
* **Nguyên lý:** "Kẻ ra lệnh là Vua".
* Hệ thống quét toàn bộ mã nguồn để vẽ sơ đồ xem **File nào gọi File nào** (thông qua lệnh `\input`, `\include`).
* **Quy tắc:** Nếu File A được gọi bởi File B $\rightarrow$ File A là **Con**, File B là **Cha**. File Con sẽ bị trừ điểm rất nặng vì nó không thể là Root.

### 3. Hệ thống Chấm điểm Đa tiêu chí (Multi-factor Scoring)

Mỗi ứng viên vượt qua vòng loại sẽ được chấm điểm dựa trên bảng quy tắc sau:

| Tiêu chí (Criteria) | Điểm số | Lý giải |
| :--- | :--- | :--- |
| **Dấu vết biên dịch (.bbl)** | `+60` | Nếu tồn tại file `.bbl` trùng tên, chứng tỏ tác giả đã từng compile file này. Đây là bằng chứng thép. |
| **Bị file khác gọi (Is Child)** | `-50` (mỗi lần) | File gốc là file khởi xướng, không bao giờ bị file khác gọi. |
| **Log file (.log)** | `+30` | Tương tự .bbl, file log trùng tên là dấu vết của lần chạy trước. |
| **Cấu trúc (`\begin{document}`)** | `+20` | Đảm bảo file có nội dung hoàn chỉnh. |
| **Chứa Bibliography** | `+15` | Bài báo khoa học luôn phải có danh mục tham khảo. |
| **Gọi file khác (Is Parent)** | `+3` (mỗi file) | File gốc thường đóng vai trò "Nhạc trưởng", kết nối các file con. |
| **Tên file chuẩn** | `+10` | Tên là `main`, `paper`, `ms`, `article`. |
| **Tên file rác/Slide** | `-50` | Tên chứa `letter`, `response` hoặc dùng `\documentclass{beamer}`. |

---

### 🧩 Cơ chế giải quyết hòa (Tie-Breaker)

Trong trường hợp hiếm hoi có 2 file điểm cao ngang nhau (ví dụ: `main_v1.tex` và `main_v2.tex`), thuật toán sẽ chọn dựa trên:
1.  **Độ ưu tiên tên:** Ưu tiên `main.tex` hơn tên lạ.
2.  **Kích thước nội dung:** File nặng hơn thường là bản đầy đủ hơn.

In [1]:
import os
import re
import json
from tqdm import tqdm

In [2]:


# Các thư mục không cần quét để tối ưu tốc độ
BLOCKLIST_DIRS = {'.git', 'images', 'figures', '__pycache__', 'node_modules', 'media'}

def build_dependency_map(folder_path):
    """
    Xây dựng bản đồ phụ thuộc: File nào bị file nào gọi?
    Trả về dict: { 'child_full_path': ['parent_full_path'] }
    """
    dependency_map = {}
    name_to_path = {}
    
    # 1. Map toàn bộ file .tex trong folder này để tra cứu nhanh
    for root, dirs, files in os.walk(folder_path):
        dirs[:] = [d for d in dirs if d not in BLOCKLIST_DIRS]
        for f in files:
            if f.lower().endswith('.tex'):
                full_path = os.path.join(root, f)
                name_to_path[f] = full_path

    # 2. Quét nội dung để tìm quan hệ cha-con
    for root, dirs, files in os.walk(folder_path):
        dirs[:] = [d for d in dirs if d not in BLOCKLIST_DIRS]
        for file in files:
            if not file.lower().endswith('.tex'): continue
            
            parent_path = os.path.join(root, file)
            try:
                with open(parent_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    # Tìm lệnh \input, \include, \subfile
                    # Regex bắt: \input{filename}
                    matches = re.findall(r'\\(?:input|include|subfile)(?:\[.*?\])?\{([^}]+)\}', content)
                    
                    for match in matches:
                        child_name = os.path.basename(match.strip())
                        if not child_name.lower().endswith('.tex'): 
                            child_name += '.tex'
                        
                        # Nếu file con tồn tại trong danh sách file của folder
                        if child_name in name_to_path:
                            child_path = name_to_path[child_name]
                            
                            # Ghi nhận: child_path bị gọi bởi parent_path
                            if child_path not in dependency_map:
                                dependency_map[child_path] = []
                            dependency_map[child_path].append(parent_path)
            except:
                pass
                
    return dependency_map

def get_score(file_path, content, context_files, dependency_map):
    """Hàm chấm điểm logic LaTeX chuẩn"""
    filename = os.path.basename(file_path)
    base_name = os.path.splitext(filename)[0]
    clean_content = re.sub(r'%.*', '', content) # Xóa comment
    
    score = 0
    
    # --- 1. CẤU TRÚC BẮT BUỘC (GATEKEEPER) ---
    # Nếu không có documentclass -> Loại ngay (Theo yêu cầu bỏ Plain TeX)
    if r'\documentclass' not in clean_content:
        return -1000 
    
    # Có documentclass là đạt yêu cầu cơ bản
    score += 20
    
    if r'\begin{document}' in clean_content:
        score += 20
        
    # --- 2. DEPENDENCY CHECK (QUAN TRỌNG) ---
    # Nếu file này bị file khác gọi -> Nó là file con -> Trừ điểm cực nặng
    if file_path in dependency_map:
        # Trừ 50 điểm cho mỗi lần bị gọi
        score -= 50 * len(dependency_map[file_path])

    # --- 3. DẤU HIỆU "VỆ TINH" (FORENSICS) ---
    # File .bbl sinh ra trùng tên file gốc -> Dấu hiệu mạnh nhất
    if f"{base_name}.bbl" in context_files: score += 60
    if f"{base_name}.bib" in context_files: score += 20
    if f"{base_name}.log" in context_files: score += 30
    
    # --- 4. NỘI DUNG ---
    if r'\bibliography' in clean_content or r'\begin{thebibliography}' in clean_content: score += 15
    if r'\maketitle' in clean_content: score += 10
    if r'\begin{abstract}' in clean_content: score += 10
    
    # Đếm số lượng file con mà nó gọi (Nhạc trưởng thường gọi nhiều nhạc công)
    input_count = len(re.findall(r'\\(input|include|subfile)\{', clean_content))
    score += min(input_count * 3, 30) # Max cộng 30 điểm

    # --- 5. HEURISTIC TÊN FILE ---
    lower_name = filename.lower()
    
    # Điểm cộng tên chuẩn
    if lower_name in ['main.tex', 'ms.tex', 'paper.tex', 'article.tex']: 
        score += 10
    
    # Điểm trừ tên file rác/template
    if any(x in lower_name for x in ['template', 'sample', 'example']) and score < 60:
        score -= 10
    if 'response' in lower_name or 'reply' in lower_name or 'letter' in lower_name:
        score -= 50
        
    # Trừ điểm file slide, standalone
    if r'\documentclass{beamer}' in clean_content: score -= 50
    if r'\documentclass{standalone}' in clean_content: score -= 20
    if r'\documentclass{letter}' in clean_content: score -= 50

    return score

def find_root_tex_file(version_folder_path):
    """
    Hàm chính: Tìm file LaTeX gốc trong thư mục version.
    Input: Đường dẫn tới folder chứa code (vd: .../tex/version1)
    Output: Đường dẫn tuyệt đối tới file main.tex (hoặc None nếu không tìm thấy)
    """
    # 1. Xây dựng bản đồ phụ thuộc trước
    dep_map = build_dependency_map(version_folder_path)
    
    # 2. Lấy danh sách toàn bộ file để check vệ tinh (.bbl, .bib)
    all_files = set()
    tex_files = []
    
    for root, dirs, files in os.walk(version_folder_path):
        dirs[:] = [d for d in dirs if d not in BLOCKLIST_DIRS]
        for f in files:
            all_files.add(f)
            if f.lower().endswith('.tex'):
                tex_files.append(os.path.join(root, f))
    
    if not tex_files:
        return None

    # 3. Chấm điểm từng ứng viên
    candidates = []
    for path in tex_files:
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                score = get_score(path, content, all_files, dep_map)
                
                # Chỉ lấy ứng viên có điểm dương hoặc ít nhất không bị loại (-1000)
                if score > -100:
                    candidates.append({
                        'path': path, 
                        'name': os.path.basename(path),
                        'score': score, 
                        'len': len(content)
                    })
        except:
            pass

    if not candidates:
        return None
    
    # 4. Sắp xếp: Ưu tiên Điểm cao -> Sau đó đến độ dài nội dung
    candidates.sort(key=lambda x: (x['score'], x['len']), reverse=True)
    
    # 5. Xử lý Tie-breaker (nếu Top 1 và Top 2 điểm bằng nhau)
    if len(candidates) >= 2:
        top1 = candidates[0]
        top2 = candidates[1]
        
        # Nếu điểm chênh lệch không đáng kể (< 5)
        if (top1['score'] - top2['score']) < 5:
            # Ưu tiên file có tên chuẩn
            prio_names = ['main.tex', 'ms.tex', 'paper.tex', 'article.tex']
            if top1['name'].lower() not in prio_names and top2['name'].lower() in prio_names:
                return top2['path']
            
            # Nếu tên cũng không giúp ích, lấy file NẶNG HƠN ĐÁNG KỂ
            if top2['len'] > top1['len'] * 1.5: 
                return top2['path']

    # Trả về đường dẫn của ứng viên số 1
    return candidates[0]['path']

In [3]:

# --- CHẠY THỬ NGHIỆM TRÊN FOLDER DATA_RAW ---

# Lưu ý: Vì notebook nằm trong folder 'notebooks', 
# nên đường dẫn sang data phải lùi lại 1 cấp (..)
data_raw_path = '../data_raw' 

all_root_files = []

if os.path.exists(data_raw_path):
    # Lấy danh sách các folder con (ví dụ: 2403-00530)
    paper_folders = [f for f in os.listdir(data_raw_path) 
                     if os.path.isdir(os.path.join(data_raw_path, f))]
    
    print(f"Tìm thấy {len(paper_folders)} bài báo. Đang quét file gốc...\n")
    
    for folder_id in paper_folders: # Chỉ chạy thử 5 bài đầu tiên
        folder_path = os.path.join(data_raw_path, folder_id, 'tex')
        print(folder_path)

        for version in os.listdir(folder_path):
            version_path =  os.path.join(folder_path, version)
            if not os.path.isdir(version_path):
                continue
            
            print(f"  Phiên bản: {version}")
            
            main_file = find_root_tex_file(version_path)

            root_file_info = {"paper_id": folder_id, "version": version, "path": main_file}
            
            if main_file:
                print(f"✅ {folder_id}: {main_file}")
                all_root_files.append(root_file_info)
            else:
                print(f"❌ {folder_id}: Không tìm thấy file gốc!")
else:
    print(f"⚠️ Không tìm thấy đường dẫn: {data_raw_path}")
    print("Hãy kiểm tra lại vị trí file notebook so với folder data_raw")

Tìm thấy 11 bài báo. Đang quét file gốc...

../data_raw\2403-00530\tex
  Phiên bản: 2403-00530v1


✅ 2403-00530: ../data_raw\2403-00530\tex\2403-00530v1\main.tex
  Phiên bản: 2403-00530v2
✅ 2403-00530: ../data_raw\2403-00530\tex\2403-00530v2\main_rerev2025.tex
../data_raw\2403-00531\tex
  Phiên bản: 2403-00531v1
✅ 2403-00531: ../data_raw\2403-00531\tex\2403-00531v1\main.tex
  Phiên bản: 2403-00531v2
✅ 2403-00531: ../data_raw\2403-00531\tex\2403-00531v2\apssamp.tex
../data_raw\2403-00532\tex
  Phiên bản: 2403-00532v1
✅ 2403-00532: ../data_raw\2403-00532\tex\2403-00532v1\file_arxiv.tex
  Phiên bản: 2403-00532v2
✅ 2403-00532: ../data_raw\2403-00532\tex\2403-00532v2\file_arxiv2.tex
../data_raw\2403-00533\tex
  Phiên bản: 2403-00533v1
✅ 2403-00533: ../data_raw\2403-00533\tex\2403-00533v1\main.tex
  Phiên bản: 2403-00533v2
✅ 2403-00533: ../data_raw\2403-00533\tex\2403-00533v2\JCAP.tex
../data_raw\2403-00534\tex
  Phiên bản: 2403-00534v1
✅ 2403-00534: ../data_raw\2403-00534\tex\2403-00534v1\1rdm.tex
../data_raw\2403-00535\tex
  Phiên bản: 2403-00535v1
✅ 2403-00535: ../data_raw\2403-00535\t

In [4]:
print(all_root_files)


[{'paper_id': '2403-00530', 'version': '2403-00530v1', 'path': '../data_raw\\2403-00530\\tex\\2403-00530v1\\main.tex'}, {'paper_id': '2403-00530', 'version': '2403-00530v2', 'path': '../data_raw\\2403-00530\\tex\\2403-00530v2\\main_rerev2025.tex'}, {'paper_id': '2403-00531', 'version': '2403-00531v1', 'path': '../data_raw\\2403-00531\\tex\\2403-00531v1\\main.tex'}, {'paper_id': '2403-00531', 'version': '2403-00531v2', 'path': '../data_raw\\2403-00531\\tex\\2403-00531v2\\apssamp.tex'}, {'paper_id': '2403-00532', 'version': '2403-00532v1', 'path': '../data_raw\\2403-00532\\tex\\2403-00532v1\\file_arxiv.tex'}, {'paper_id': '2403-00532', 'version': '2403-00532v2', 'path': '../data_raw\\2403-00532\\tex\\2403-00532v2\\file_arxiv2.tex'}, {'paper_id': '2403-00533', 'version': '2403-00533v1', 'path': '../data_raw\\2403-00533\\tex\\2403-00533v1\\main.tex'}, {'paper_id': '2403-00533', 'version': '2403-00533v2', 'path': '../data_raw\\2403-00533\\tex\\2403-00533v2\\JCAP.tex'}, {'paper_id': '2403-00

## Gộp các file tex lại

Sau khi tìm được file root, thì ta tiếp tục dùng regex để gộp các file con vào file gốc, tạo thành một file LaTeX hoàn chỉnh. Quá trình này sẽ lặp lại đệ quy cho đến khi không còn lệnh `\input` hay `\include` nào nữa.

Sau khi đã xác định được các file, ta sẽ gộp lại bằng cách dfs để gộp sâu hơn, sau khi duyệt hết 1 file thì ta kẹp giữa 2 flag là 

```
%--- BEGIN FILE: filename.tex ---
%--- END FILE: filename.tex ---

```

In [5]:

class LatexFlattener:
    def __init__(self, root_file_path, paper_id, version):
        self.root_path = os.path.abspath(root_file_path)
        self.root_dir = os.path.dirname(self.root_path)
        self.paper_id = paper_id
        self.version = version
        print(f"📝 Khởi tạo LatexFlattener cho Paper: {self.paper_id}, Version: {self.version}")
        self.merged_files = [] # Danh sách các file đã gộp thành công
        self.missing_files = [] # Danh sách các file bị thiếu

    def flatten(self):
        """
        Hàm chính: Thực hiện gộp và trả về cấu trúc Dictionary (JSON object)
        """
        # Bắt đầu đệ quy từ root
        full_content = self._process_file(self.root_path)
        
        # Tạo object kết quả
        result_object = {
            "paper_id": self.paper_id,
            "version": self.version,
            "root_file_path": self.root_path,
            "metadata": {
                "total_length": len(full_content),
                "merged_count": len(self.merged_files),
                "merged_files": self.merged_files,
                "missing_files": self.missing_files
            },
            "content": full_content
        }
        return result_object

    def _read_file(self, path):
        if not os.path.exists(path): return None
        try:
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                return f.read()
        except: return None

    def _remove_comments(self, text):
        """Xóa comment gốc của tác giả để giảm nhiễu, nhưng giữ lại marker của mình sau này"""
        # Regex: Tìm ký tự % không đi sau dấu \
        return re.sub(r'(?<!\\)%.*', '', text)

    def _remove_bibliography(self, text):
        """Loại bỏ phần tài liệu tham khảo theo yêu cầu"""
        text = re.sub(r'\\bibliography\{[^}]+\}', '', text)
        text = re.sub(r'\\printbibliography', '', text)
        text = re.sub(r'\\begin\{thebibliography\}.*?\\end\{thebibliography\}', '', text, flags=re.DOTALL)
        return text

    def _process_file(self, current_path, visited=None):
        if visited is None: visited = set()
        
        abs_path = os.path.abspath(current_path)
        rel_path = os.path.relpath(abs_path, self.root_dir).replace('\\', '/') # Chuẩn hóa đường dẫn
        
        # 1. Check vòng lặp
        if abs_path in visited:
            return f"\n% <WARNING: Circular dependency detected for {rel_path}>\n"
        visited.add(abs_path)

        # 2. Đọc nội dung
        raw_content = self._read_file(abs_path)
        if raw_content is None:
            self.missing_files.append(rel_path)
            return f"\n% <WARNING: File not found: {rel_path}>\n"
        
        self.merged_files.append(rel_path)

        # 3. Làm sạch sơ bộ (Xóa comment gốc + Xóa Bib)
        content = self._remove_comments(raw_content)
        content = self._remove_bibliography(content)

        # 4. Tìm và thay thế đệ quy các file con
        # Regex hỗ trợ: \input{file}, \include{file}, \subfile{file}, \input file
        pattern = re.compile(r'\\(?:input|include|subfile)(?:(?:\s*\{([^}]+)\})|(?:\s+([^\s%]+)))')

        def replace_match(match):
            fname = match.group(1) or match.group(2)
            if not fname: return ""
            fname = fname.strip()
            if not fname.lower().endswith('.tex'): fname += '.tex'
            
            # Resolve path
            child_path = os.path.join(self.root_dir, fname)
            if not os.path.exists(child_path):
                child_path = os.path.join(os.path.dirname(abs_path), fname)
            
            # Đệ quy
            child_content = self._process_file(child_path, visited)
            
            # QUAN TRỌNG: Kẹp nội dung giữa 2 Marker
            return (f"\n% <BEGIN_FILE: {fname}>\n"
                    f"{child_content}"
                    f"\n% <END_FILE: {fname}>\n")

        flattened_content = pattern.sub(replace_match, content)
        
        return flattened_content

In [6]:


# --- HÀM TÍCH HỢP CHO TOÀN BỘ DATASET ---
def process_and_save_dataset(root_file_info_list, output_json_path):
    """
    Input: Danh sách các root file tìm được từ bước trước (list of dict)
    Output: Lưu 1 file JSONL (mỗi dòng là 1 json object của 1 bài báo)
    """
    print(f"🚀 Bắt đầu Flattening {len(root_file_info_list)} versions...")
    
    with open(output_json_path, 'w', encoding='utf-8') as f_out:
        for info in tqdm(root_file_info_list, desc="Flattening"):
            root_path = info['path']
            # print(info)
            try:
                flattener = LatexFlattener(root_path, paper_id=info['paper_id'], version=info['version'])
                result = flattener.flatten()                
                f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
            except Exception as e:
                print(f"Lỗi khi xử lý {root_path}: {e}")

    print(f"✅ Hoàn tất! Dữ liệu đã lưu tại: {output_json_path}")

In [7]:
# Giả sử 'all_root_files' là list chứa kết quả tìm được từ bước trước
# Ví dụ: all_root_files = [{'path': 'data/2403-00611/tex/v1/main.tex', ...}, ...]

# Nếu bạn chưa có list này, hãy chạy lại hàm find_root_in_version cho toàn bộ dataset và gom vào 1 list

OUTPUT_FILE = "flattened_papers.jsonl"

# Chạy xử lý hàng loạt
if 'all_root_files' in globals() and all_root_files:
    process_and_save_dataset(all_root_files, OUTPUT_FILE)
else:
    print("⚠️ Hãy đảm bảo bạn đã có biến 'all_root_files' chứa danh sách các file gốc.")
    
    # Test thử với 1 file cụ thể
    # sample_root = r"ĐƯỜNG_DẪN_ĐẾN_FILE_ROOT_THẬT_CỦA_BẠN"
    # flattener = LatexFlattener(sample_root)
    # print(json.dumps(flattener.flatten(), indent=2)[:1000]) # In thử 1000 ký tự đầu

🚀 Bắt đầu Flattening 19 versions...


Flattening: 100%|██████████| 19/19 [00:00<00:00, 431.04it/s]

📝 Khởi tạo LatexFlattener cho Paper: 2403-00530, Version: 2403-00530v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00530, Version: 2403-00530v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00531, Version: 2403-00531v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00531, Version: 2403-00531v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00532, Version: 2403-00532v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00532, Version: 2403-00532v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00533, Version: 2403-00533v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00533, Version: 2403-00533v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00534, Version: 2403-00534v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00535, Version: 2403-00535v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00536, Version: 2403-00536v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00537, Version: 2403-00537v1
📝 Khởi tạo LatexFlattener cho Paper: 2403-00537, Version: 2403-00537v2
📝 Khởi tạo LatexFlattener cho Paper: 2403-00538, Version: 2403-00538v1
📝 Khởi

## Chạy thử nghiệm tính đúng đắn của thuật toán gộp file trên

In [8]:
import os
import shutil
import tempfile
import json
import re

# ==========================================
# 1. SETUP: TẠO MÊ CUNG FILE (COMPLEX STRUCTURE)
# ==========================================

def create_complex_test_env():
    # Tạo thư mục tạm
    base_dir = tempfile.mkdtemp()
    
    # Cấu trúc file sẽ tạo:
    # base_dir/
    # ├── main.tex                  (ROOT)
    # ├── common_macros.tex         (Được gọi từ folder con bằng ../)
    # ├── references.bib            (File rác, cần bị ignore)
    # └── chapters/
    #     ├── intro.tex             (Level 1)
    #     └── deep_logic/
    #         ├── theory.tex        (Level 2)
    #         └── components/
    #             └── equation.tex  (Level 3 - Sâu nhất)

    files = {
        # ROOT FILE
        "main.tex": r"""
        \documentclass{article}
        \usepackage{import}
        \begin{document}
        \section{Main Title}
        This is the root file.
        
        % Test xóa comment: Dòng này không được xuất hiện trong output.
        
        % Trap 1: Gọi file trong folder con
        \input{chapters/intro}
        
        \section{Conclusion}
        End of main.
        \bibliography{references} % Trap 2: Phải xóa dòng này
        \end{document}
        """,

        # FILE NẰM NGOÀI (Dùng để test ../)
        "common_macros.tex": r"""
        \def\mycommand{Macro Loaded Successfully}
        """,

        # LEVEL 1
        "chapters/intro.tex": r"""
        \section{Introduction}
        Intro content here.
        
        % Trap 3: Gọi sâu hơn (Level 2)
        \include{deep_logic/theory}
        """,

        # LEVEL 2
        "chapters/deep_logic/theory.tex": r"""
        \subsection{Theoretical Framework}
        Theory content.
        
        % Trap 4: Gọi ngược lên thư mục cha của root (Relative path tricky)
        \input{../../common_macros.tex}
        
        % Trap 5: Gọi sâu nữa (Level 3)
        \subfile{components/equation}
        """,

        # LEVEL 3
        "chapters/deep_logic/components/equation.tex": r"""
        \begin{equation}
            E = mc^2
        \end{equation}
        Final deepest content.
        """
    }

    print(f"📂 Đang khởi tạo môi trường test tại: {base_dir}")
    for rel_path, content in files.items():
        abs_path = os.path.join(base_dir, rel_path)
        os.makedirs(os.path.dirname(abs_path), exist_ok=True)
        with open(abs_path, "w", encoding="utf-8") as f:
            f.write(content.strip())
            
    return base_dir, os.path.join(base_dir, "main.tex")

# ==========================================
# 2. EXECUTION: CHẠY FLATTENER
# ==========================================

def visualize_structure(content):
    """
    Hàm này parse các marker <BEGIN_FILE...> để vẽ lại cây cấu trúc
    Giúp mắt thường kiểm tra xem việc lồng ghép có đúng thứ tự không.
    """
    lines = content.split('\n')
    indent_level = 0
    tree_visualization = []
    
    print("\n🌳 VISUALIZATION (CẤU TRÚC SAU KHI GỘP):")
    print("="*50)
    
    for line in lines:
        # Nếu gặp Marker Begin -> Tăng thụt đầu dòng
        match_begin = re.search(r'% <BEGIN_FILE: (.*?)>', line)
        if match_begin:
            fname = match_begin.group(1)
            print(f"{'|   ' * indent_level}├── 📂 MỞ FILE: {fname}")
            indent_level += 1
            continue
            
        # Nếu gặp Marker End -> Giảm thụt đầu dòng
        match_end = re.search(r'% <END_FILE: (.*?)>', line)
        if match_end:
            indent_level -= 1
            fename = match_end.group(1)
            print(f"{'|   ' * indent_level}└── 🏁 ĐÓNG FILE: {fename}")
            continue
            
        # In nội dung text (đại diện)
        if line.strip() and not line.strip().startswith('%'):
            # Cắt ngắn nội dung để dễ nhìn
            preview = (line.strip()[:40] + '...') if len(line.strip()) > 40 else line.strip()
            print(f"{'|   ' * indent_level}|  \"{preview}\"")

    print("="*50)

# ==========================================
# 3. MAIN RUNNER
# ==========================================

try:
    # 1. Tạo Lab
    temp_dir, root_file = create_complex_test_env()
    
    # 2. Gọi Class của bạn (Giả sử class LatexFlattener đã được define ở cell trước)
    # Nếu chưa có class, code sẽ báo lỗi NameError
    flattener = LatexFlattener(root_file, 'test-id', 'test-version')
    result = flattener.flatten()
    
    # 3. Kiểm tra kết quả
    print("\n✅ KẾT QUẢ FLATTEN JSON:")
    print(f"- Root File: {result['root_file_path']}")
    print(f"- Tổng độ dài: {result['metadata']['total_length']} chars")
    print(f"- Files đã gộp: {json.dumps(result['metadata']['merged_files'], indent=2)}")
    
    if result['metadata']['missing_files']:
        print(f"⚠️ Files bị thiếu: {result['metadata']['missing_files']}")
    else:
        print("- Không có file nào bị thiếu.")

    # 4. Kiểm tra logic loại bỏ rác
    content = result['content']
    if "Test xóa comment" in content:
        print("❌ LỖI: Comment chưa được xóa!")
    else:
        print("✅ PASS: Comment đã được xóa.")
        
    if "\\bibliography" in content:
        print("❌ LỖI: Bibliography chưa được xóa!")
    else:
        print("✅ PASS: Bibliography đã được xóa.")

    # 5. Vẽ cây để người dùng verify
    visualize_structure(content)

finally:
    # Dọn dẹp
    if 'temp_dir' in globals() and os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print("\n🧹 Đã dọn dẹp môi trường test.")

📂 Đang khởi tạo môi trường test tại: C:\Users\duyla\AppData\Local\Temp\tmpjbn3gwu0
📝 Khởi tạo LatexFlattener cho Paper: test-id, Version: test-version

✅ KẾT QUẢ FLATTEN JSON:
- Root File: C:\Users\duyla\AppData\Local\Temp\tmpjbn3gwu0\main.tex
- Tổng độ dài: 900 chars
- Files đã gộp: [
  "main.tex",
  "chapters/intro.tex",
  "chapters/deep_logic/theory.tex",
  "common_macros.tex",
  "chapters/deep_logic/components/equation.tex"
]
- Không có file nào bị thiếu.
✅ PASS: Comment đã được xóa.
✅ PASS: Bibliography đã được xóa.

🌳 VISUALIZATION (CẤU TRÚC SAU KHI GỘP):
|  "\documentclass{article}"
|  "\usepackage{import}"
|  "\begin{document}"
|  "\section{Main Title}"
|  "This is the root file."
├── 📂 MỞ FILE: chapters/intro.tex
|   |  "\section{Introduction}"
|   |  "Intro content here."
|   ├── 📂 MỞ FILE: deep_logic/theory.tex
|   |   |  "\subsection{Theoretical Framework}"
|   |   |  "Theory content."
|   |   ├── 📂 MỞ FILE: ../../common_macros.tex
|   |   |   |  "\def\mycommand{Macro Loade

Kết luận: Thuật toán gộp file hoạt động chính xác, các file con được chèn đúng vị trí trong file gốc, và cấu trúc tài liệu LaTeX được duy trì nguyên vẹn.

In [9]:
import re
import uuid

class LatexStructureBuilder:
    def __init__(self, flattened_content, paper_id, version):
        self.content = flattened_content
        self.paper_id = paper_id
        self.version = version
        # Định nghĩa thứ tự cấp bậc (nhỏ hơn là cấp cao hơn/cha)
        self.HIERARCHY_LEVELS = {
            'document': 0,      # Root
            'part': 1,
            'chapter': 2,
            'section': 3,
            'subsection': 4,
            'subsubsection': 5,
            'paragraph': 6,
            'subparagraph': 7
        }
        # Regex để bắt các header: \section{Title}, \section*{Title}, \chapter{...}
        # Group 1: command (section, chapter...)
        # Group 2: * (nếu có)
        # Group 3: Title
        self.SECTION_REGEX = re.compile(
            r'\\(part|chapter|section|subsection|subsubsection|paragraph|subparagraph)(\*?)\s*\{([^}]+)\}',
            re.IGNORECASE
        )

    def build_coarse_tree(self):
        """
        Xây dựng cây cấu trúc thô.
        Trả về node gốc (Document) chứa toàn bộ cây con.
        """
        # 1. Tạo Node gốc (Document) [cite: 584]
        root = {
            'id': f'{self.paper_id}-{version}-document-{uuid.uuid4()}',
            'type': 'document',
            'title': 'Root Document',
            'level': 0,
            'raw_content': "", # Nội dung mở đầu trước khi vào section đầu tiên (ví dụ abstract)
            'children': []
        }

        # Stack để theo dõi cha hiện tại. Khởi đầu chỉ có root.
        # Stack store: reference tới các node dictionary
        stack = [root]

        # 2. Tìm tất cả các vị trí section headers
        matches = list(self.SECTION_REGEX.finditer(self.content))
        
        # Biến cursor để theo dõi vị trí cắt text
        cursor = 0

        for match in matches:
            start_idx = match.start()
            end_idx = match.end()
            
            command = match.group(1)
            is_starred = match.group(2) == '*'
            title = match.group(3).strip()
            
            # Lấy level của command hiện tại
            current_level = self.HIERARCHY_LEVELS.get(command, 100)

            # --- BƯỚC A: GÁN NỘI DUNG CHO NODE ĐANG ACTIVE ---
            # Text từ con trỏ cũ đến đầu header mới thuộc về node đang nằm trên đỉnh stack
            # (tức là node cha hoặc anh em liền trước)
            text_segment = self.content[cursor:start_idx]
            if text_segment.strip():
                # Gán vào content của node hiện tại trên đỉnh stack
                # Lưu ý: Ở Part 1, ta gộp thô, chưa chia nhỏ sentences
                if 'raw_content' not in stack[-1]:
                    stack[-1]['raw_content'] = ""
                stack[-1]['raw_content'] += text_segment

            # --- BƯỚC B: TÌM CHA CHO NODE MỚI (Adjust Stack) ---
            # Logic: Pop stack cho đến khi gặp node có level nhỏ hơn (cao hơn) node hiện tại
            # Ví dụ: Đang ở Subsection (4), gặp Section (3) -> Pop Subsection ra, để Section mới làm con của Chapter (2)
            # Nếu gặp Subsection khác (4) -> Pop Subsection cũ ra, Subsection mới thay thế vị trí đó (anh em).
            
            while len(stack) > 1 and stack[-1]['level'] >= current_level:
                stack.pop()
            
            # Parent là node đang nằm trên đỉnh stack sau khi pop
            parent = stack[-1]

            # --- BƯỚC C: TẠO NODE MỚI ---
            # Lưu ý: Bỏ qua phần References/Bibliography nếu tiêu đề match 
            if title.lower() in ['references', 'bibliography', 'tài liệu tham khảo']:
                # Update cursor để bỏ qua header này, nhưng cần cẩn thận phần content bên dưới
                # Với logic này, content bên dưới References sẽ bị gán vào node cha của references
                # Tạm thời ta vẫn tạo node để chứa, sau này filter bỏ sau.
                pass 

            new_node = {
                # 'id': self.paper_id + f'-{command}-' + str(uuid.uuid4()),
                'id': f'{self.paper_id}-{version}-{command}-{uuid.uuid4()}',
                'type': command,
                'title': title,
                'level': current_level,
                'raw_content': "", # Sẽ được điền ở vòng lặp kế tiếp hoặc sau vòng lặp
                'children': []
            }

            # Link vào parent
            parent['children'].append(new_node)
            
            # Push node mới vào stack để nó hứng content tiếp theo
            stack.append(new_node)
            
            # Cập nhật cursor = vị trí kết thúc của header vừa xử lý
            cursor = end_idx

        # --- BƯỚC D: XỬ LÝ PHẦN DƯ CUỐI CÙNG ---
        # Text từ header cuối cùng đến hết file thuộc về node đang nằm trên đỉnh stack
        remaining_text = self.content[cursor:]
        if remaining_text.strip():
             if 'raw_content' not in stack[-1]:
                    stack[-1]['raw_content'] = ""
             stack[-1]['raw_content'] += remaining_text

        return root

    def print_tree(self, node, indent=0):
        """Hàm helper để in cây ra console kiểm tra"""
        prefix = "  " * indent
        preview = (node.get('raw_content', '')[:50] + '...') if node.get('raw_content') else "[Empty]"
        print(f"{prefix}- [{node['type'].upper()}] {node['title']} (ID: {node['id'][:8]})")
        print(f"{prefix}  Content Preview: {preview}")
        
        for child in node['children']:
            self.print_tree(child, indent + 1)
    
    def print_tree_to_file(self, root_node, output_path):
        """
        In cây ra file JSON với danh sách nodes và edges
        
        Args:
            root_node: Node gốc của cây
            output_path: Đường dẫn file JSON để lưu
        
        Output format:
        {
            "nodes": [
                {
                    "id": "uuid",
                    "type": "section",
                    "title": "Section Title",
                    "level": 3,
                    "content": "raw content..."
                }
            ],
            "edges": [
                {
                    "from": "parent_id",
                    "to": "child_id"
                }
            ]
        }
        """
        nodes = []
        edges = []
        
        def traverse(node, parent_id=None):
            # Thêm node hiện tại vào danh sách
            node_data = {
                "id": node['id'],
                "type": node['type'],
                "title": node['title'],
                "level": node['level'],
                "content": node.get('raw_content', '')
            }
            nodes.append(node_data)
            
            # Nếu có parent, tạo edge
            if parent_id is not None:
                edges.append({
                    "from": parent_id,
                    "to": node['id']
                })
            
            # Đệ quy cho các children
            for child in node.get('children', []):
                traverse(child, node['id'])
        
        # Bắt đầu traverse từ root
        traverse(root_node)
        
        # Tạo cấu trúc dữ liệu cuối cùng
        output_data = {
            "nodes": nodes,
            "edges": edges,
            "metadata": {
                "total_nodes": len(nodes),
                "total_edges": len(edges)
            }
        }
        
        # Ghi ra file
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Đã lưu cấu trúc cây vào: {output_path}")
        print(f"   - Tổng số nodes: {len(nodes)}")
        print(f"   - Tổng số edges: {len(edges)}")


In [10]:

# --- VÍ DỤ SỬ DỤNG VỚI CLASS CŨ ---
# Giả sử bạn đã có biến `full_content` từ LatexFlattener
flattened_content = flattener.flatten()['content']
print(flattened_content)
# builder = LatexStructureBuilder(flattened_content)
# root_tree = builder.build_coarse_tree()
# builder.print_tree(root_tree)


% <WARNING: File not found: main.tex>



In [11]:
# Đọc file flattened_papers.jsonl và test LatexStructureBuilder

import json

def load_flattened_papers(jsonl_path):
    """
    Đọc file JSONL và trả về list các paper objects
    """
    papers = []
    try:
        with open(jsonl_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():  # Bỏ qua dòng trống
                    paper = json.loads(line)
                    papers.append(paper)
        print(f"✅ Đã load {len(papers)} papers từ {jsonl_path}")
        return papers
    except FileNotFoundError:
        print(f"❌ Không tìm thấy file: {jsonl_path}")
        return []
    except Exception as e:
        print(f"❌ Lỗi khi đọc file: {e}")
        return []

# Load papers
papers = load_flattened_papers("flattened_papers.jsonl")

# Hiển thị thông tin tổng quan
if papers:
    print("\n📊 THÔNG TIN TỔNG QUAN:")
    print("="*60)
    for i, paper in enumerate(papers, 1):
        print(f"{i}. Paper ID: {paper['paper_id']} - Version: {paper['version']}")
        print(f"   - Content length: {paper['metadata']['total_length']:,} chars")
        print(f"   - Merged files: {paper['metadata']['merged_count']}")
        if paper['metadata']['missing_files']:
            print(f"   - Missing files: {len(paper['metadata']['missing_files'])}")
    print("="*60)

✅ Đã load 19 papers từ flattened_papers.jsonl

📊 THÔNG TIN TỔNG QUAN:
1. Paper ID: 2403-00530 - Version: 2403-00530v1
   - Content length: 41,291 chars
   - Merged files: 1
2. Paper ID: 2403-00530 - Version: 2403-00530v2
   - Content length: 51,454 chars
   - Merged files: 1
3. Paper ID: 2403-00531 - Version: 2403-00531v1
   - Content length: 62,735 chars
   - Merged files: 1
4. Paper ID: 2403-00531 - Version: 2403-00531v2
   - Content length: 72,185 chars
   - Merged files: 1
5. Paper ID: 2403-00532 - Version: 2403-00532v1
   - Content length: 179,231 chars
   - Merged files: 1
6. Paper ID: 2403-00532 - Version: 2403-00532v2
   - Content length: 180,119 chars
   - Merged files: 1
7. Paper ID: 2403-00533 - Version: 2403-00533v1
   - Content length: 64,852 chars
   - Merged files: 1
8. Paper ID: 2403-00533 - Version: 2403-00533v2
   - Content length: 64,048 chars
   - Merged files: 1
9. Paper ID: 2403-00534 - Version: 2403-00534v1
   - Content length: 59,864 chars
   - Merged files: 2
1

In [12]:
# Test LatexStructureBuilder với paper đầu tiên

if papers:
    # Lấy paper đầu tiên
    test_paper = papers[0]
    print(f"🧪 Testing với Paper: {test_paper['paper_id']} - {test_paper['version']}")
    print(f"📄 Content length: {len(test_paper['content']):,} characters\n")
    
    # Build cây cấu trúc với paper_id và version
    builder = LatexStructureBuilder(test_paper['content'], 
                                   paper_id=test_paper['paper_id'],
                                   version=test_paper['version'])
    root_tree = builder.build_coarse_tree()
    
    print("\n🌳 CẤU TRÚC CÂY (TREE STRUCTURE):")
    print("="*70)
    builder.print_tree(root_tree)
    print("="*70)
    

    print(f"\n📌 Paper Info:")
    print(f"   - Paper ID: {test_paper['paper_id']}")
    print(f"   - Version: {test_paper['version']}")
    
    # Lưu cấu trúc cây vào file JSON
    output_tree_path = f"structure_{test_paper['paper_id']}_{test_paper['version']}.json"
    builder.print_tree_to_file(root_tree, output_tree_path)
    
    # Đếm số lượng sections


    def count_nodes(node):
        count = 1
        for child in node.get('children', []):
            count += count_nodes(child)
        return count
    
    total_nodes = count_nodes(root_tree)
    print(f"\n📊 Tổng số nodes: {total_nodes}")
    print(f"📊 Số children của root: {len(root_tree['children'])}")
else:
    print("⚠️ Không có papers để test. Hãy chạy lại các cell trước để tạo file flattened_papers.jsonl")

🧪 Testing với Paper: 2403-00530 - 2403-00530v1
📄 Content length: 41,291 characters


🌳 CẤU TRÚC CÂY (TREE STRUCTURE):
- [DOCUMENT] Root Document (ID: 2403-005)
  Content Preview: \documentclass[
twocolumn,
superscriptaddress,
ams...
  - [SECTION] Introduction (ID: 2403-005)
    Content Preview: \label{intro}

Nearly flat bands confined within a...
  - [SECTION] Extended Hubbard corrected tight-binding model (ID: 2403-005)
    Content Preview: \label{sec2}
...
    - [SUBSECTION] Interacting correction (ID: 2403-005)
      Content Preview: \label{sec2-1}

\begin{figure}[tb!] 
\includegraph...
    - [SUBSECTION] Rhombohedral stacked N-layer graphene (ID: 2403-005)
      Content Preview: \label{rhomboG}

\begin{table}[tb!] 
\begin{threep...
  - [SECTION] Realistic gapped states and Flatbands (ID: 2403-005)
    Content Preview: \label{sec3}

\begin{figure*}[bt!] 
\subfloat[\lab...
  - [SECTION] Conclusion (ID: 2403-005)
    Content Preview: \label{conclusion}

In this paper, we introduced t

In [13]:
# Test với nhiều papers và so sánh cấu trúc

if len(papers) > 1:
    print("📊 SO SÁNH CẤU TRÚC GIỮA CÁC PAPERS:")
    print("="*70)
    
    for i, paper in enumerate(papers[:5], 1):  # Test 5 papers đầu tiên
        print(f"\n{i}. Paper {paper['paper_id']} - {paper['version']}")
        print(f"   Content: {len(paper['content']):,} chars")
        
        builder = LatexStructureBuilder(paper['content'],
                                       paper_id=paper['paper_id'],
                                       version=paper['version'])
        root_tree = builder.build_coarse_tree()
        
        # Đếm từng loại section
        section_counts = {}
        def count_sections(node):
            node_type = node['type']
            section_counts[node_type] = section_counts.get(node_type, 0) + 1
            for child in node.get('children', []):
                count_sections(child)
        
        count_sections(root_tree)
        
        # Hiển thị thống kê
        for section_type in ['part', 'chapter', 'section', 'subsection', 'subsubsection']:
            if section_type in section_counts:
                print(f"   - {section_type.capitalize()}: {section_counts[section_type]}")
    
    print("="*70)
else:
    print("ℹ️ Chỉ có 1 paper, bỏ qua so sánh")

📊 SO SÁNH CẤU TRÚC GIỮA CÁC PAPERS:

1. Paper 2403-00530 - 2403-00530v1
   Content: 41,291 chars
   - Section: 5
   - Subsection: 2

2. Paper 2403-00530 - 2403-00530v2
   Content: 51,454 chars
   - Section: 8
   - Subsection: 3

3. Paper 2403-00531 - 2403-00531v1
   Content: 62,735 chars
   - Section: 6
   - Subsection: 3
   - Subsubsection: 2

4. Paper 2403-00531 - 2403-00531v2
   Content: 72,185 chars
   - Section: 6
   - Subsection: 5
   - Subsubsection: 2

5. Paper 2403-00532 - 2403-00532v1
   Content: 179,231 chars
   - Section: 11
   - Subsection: 10
   - Subsubsection: 8


In [14]:
import re
import uuid

class LatexContentProcessor:
    def __init__(self, paper_id, version):
        self.paper_id = paper_id
        self.version = version
        
        # --- REGEX PATTERNS ---
        
        # 1. Block Math: $$...$$, \[...\], \begin{equation}...
        self.REGEX_MATH_BLOCK = re.compile(
            r'(\\begin\{equation\*?\}.*?\\end\{equation\*?\}|\\\[.*?\\\]|\$\$.*?\$\$)', 
            re.DOTALL
        )
        
        # 2. Figures/Tables: \begin{figure/table}...
        self.REGEX_FIGURE = re.compile(
            r'(\\begin\{(figure|table)\*?\}.*?\\end\{(figure|table)\*?\})', 
            re.DOTALL | re.IGNORECASE
        )
        
        # 3. Lists: \begin{itemize/enumerate}...
        self.REGEX_LIST = re.compile(
            r'(\\begin\{(itemize|enumerate)\}.*?\\end\{(itemize|enumerate)\})', 
            re.DOTALL
        )
        
        # 4. Sentence Splitter: Tìm dấu chấm/hỏi/thán kết thúc câu
        # Xử lý các trường hợp đặc biệt: abbreviations, số thập phân, trích dẫn
        
        # Danh sách abbreviations phổ biến trong paper khoa học
        abbrev_pattern = r'(?:Fig|Eq|Eqs|Tab|Sec|Ref|Vol|No|Ch|Dr|Prof|Ph\.D|' \
                        r'et al|i\.e|e\.g|vs|cf|etc|approx|ca|viz)'
        
        # Pattern chính:
        # - Không phải sau abbreviations
        # - Không phải giữa chữ cái đơn (U.S.)
        # - Không phải số thập phân (3.14)
        # - Cho phép dấu ngoặc kép/đơn sau dấu câu
        self.REGEX_SENTENCE = re.compile(
                    r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s+(?=[A-Z\(])'
                )

    def process_tree(self, node):
        """
        Duyệt đệ quy cây cấu trúc thô để "mổ xẻ" raw_content thành các elements.
        """
        # 1. Xử lý raw_content của node hiện tại (nếu có)
        if node.get('raw_content') and node['raw_content'].strip():
            # Tách nội dung thành các node con chi tiết (câu, hình, công thức...)
            fine_grained_nodes = self.parse_content_blocks(node['raw_content'])
            
            # QUAN TRỌNG: Chèn các node nội dung vào ĐẦU danh sách children
            # Lý do: Trong LaTeX, text của Section luôn nằm trước Subsection con.
            node['children'] = fine_grained_nodes + node['children']
            
            # Xóa raw_content để giải phóng bộ nhớ và đánh dấu là đã xử lý
            del node['raw_content']

        # 2. Đệ quy xử lý các con (bao gồm cả các Subsection cũ và các List mới tạo)
        # Lưu ý: Ta chỉ đệ quy vào các node cấu trúc (part, chapter, section...) 
        # hoặc list, không cần đệ quy vào sentence/equation (node lá).
        for child in node['children']:
            # Chỉ đệ quy nếu node con đó có thể chứa content con (ví dụ List hoặc Section con)
            if child['type'] not in ['sentence', 'equation', 'figure', 'list_item']:
                self.process_tree(child)

    def parse_content_blocks(self, text):
        """
        Cắt chuỗi text hỗn hợp thành danh sách các Node Elements
        """
        elements = []
        
        # Pattern tổng hợp để split: Math OR Figure OR List
        pattern = re.compile(
            f"({self.REGEX_MATH_BLOCK.pattern}|{self.REGEX_FIGURE.pattern}|{self.REGEX_LIST.pattern})",
            re.DOTALL | re.IGNORECASE
        )
        
        # Split text, giữ lại delimiter (chính là nội dung block)
        parts = pattern.split(text)
        
        for part in parts:
            if not part: continue
            part = part.strip()
            if not part: continue

            # --- PHÂN LOẠI & TẠO NODE ---
            
            # 1. Math Block
            if self.REGEX_MATH_BLOCK.fullmatch(part):
                elements.append(self._create_node(
                    type_name='equation',
                    title='Equation Block',
                    raw_content=self._normalize_math(part)
                ))
            
            # 2. Figure/Table
            elif self.REGEX_FIGURE.fullmatch(part):
                elements.append(self._create_node(
                    type_name='figure',
                    title='Figure/Table',
                    raw_content=self._clean_latex(part)
                ))
            
            # 3. List (Itemize/Enumerate) -> Tạo cấu trúc lồng nhau
            elif self.REGEX_LIST.fullmatch(part):
                list_node = self._process_list_block(part)
                elements.append(list_node)
            
            # 4. Text thuần -> Tách thành Sentence Nodes
            else:
                sentences = self._split_sentences(part)
                for sent in sentences:
                    elements.append(self._create_node(
                        type_name='sentence',
                        title=sent[:30] + "...", # Title xem trước
                        raw_content=self._clean_latex(sent)
                    ))
                    
        return elements

    def _process_list_block(self, list_content):
        """Xử lý riêng cho Itemize/Enumerate để tách các \item"""
        # 1. Xác định loại list (itemize hay enumerate)
        # Group 1 sẽ bắt được tên môi trường (itemize/enumerate)
        match = re.match(r'\\begin\{(itemize|enumerate)\}', list_content, re.IGNORECASE)
        list_type = match.group(1) if match else "itemize"

        list_node = self._create_node(
            type_name='list',
            title=f'List ({list_type})',
            raw_content="" 
        )
        
        # 2. Bóc vỏ (Unwrap) an toàn
        # Xóa thẻ mở đầu tiên (chỉ xóa 1 lần - count=1)
        # Regex bắt: \begin{type} theo sau có thể là [options]
        content_inner = re.sub(r'^\\begin\{' + list_type + r'\}(\[.*?\])?', '', list_content, count=1, flags=re.IGNORECASE).strip()
        
        # Xóa thẻ đóng cuối cùng (Neo vào cuối chuỗi $)
        content_inner = re.sub(r'\\end\{' + list_type + r'\}\s*$', '', content_inner, count=1, flags=re.IGNORECASE).strip()

        # 3. Tách các \item
        # Lúc này content_inner chỉ còn nội dung ruột, các list con (nếu có) vẫn nguyên vẹn
        items = re.split(r'\\item\s+', content_inner)
        
        for item in items:
            # Bỏ qua phần text rác trước \item đầu tiên (thường là khoảng trắng)
            if not item.strip(): 
                continue
                
            # Clean nội dung item
            # Lưu ý: KHÔNG dùng replace \end nữa vì ta đã bóc vỏ ở bước 2 rồi
            clean_content = self._clean_latex(item)
            
            if clean_content:
                item_node = self._create_node(
                    type_name='list_item',
                    title='List Item',
                    raw_content=clean_content
                )
                list_node['children'].append(item_node)
                
        return list_node
    def _create_node(self, type_name, title, raw_content):
        """Helper tạo node chuẩn theo format ID của bạn"""
        return {
            'id': f'{self.paper_id}-{self.version}-{type_name}-{uuid.uuid4()}',
            'type': type_name,
            'title': title,
            'level': 99, # Level thấp nhất (lá)
            'raw_content': raw_content,
            'children': []
        }

    def _split_sentences(self, text):
        """Tách câu"""
        text = re.sub(r'\s+', ' ', text) # Gộp newline thành space
        sentences = self.REGEX_SENTENCE.split(text)
        return [s.strip() for s in sentences if s.strip()]

    def _normalize_math(self, content):
        """Chuẩn hóa toán học: Convert $$ -> equation"""
        if content.startswith('$$') and content.endswith('$$'):
            inner = content[2:-2].strip()
            return f"\\begin{{equation}}{inner}\\end{{equation}}"
        elif content.startswith(r'\['):
            inner = content[2:-2].strip()
            return f"\\begin{{equation}}{inner}\\end{{equation}}"
        return content

    def _clean_latex(self, text):
        """Xóa các lệnh format rác"""
        # Xóa \centering, \hfill, label, cite, ref... tùy nhu cầu
        text = re.sub(r'\\(centering|hfill|vfill|noindent|small|tiny|large)', '', text)
        # Xóa optional params [htbp] của figure
        text = re.sub(r'\\begin\{(figure|table)\}\[.*?\]', r'\\begin{\1}', text)
        return text.strip()

In [15]:
# Test LatexContentProcessor với paper đầu tiên
paper_to_test = 1


if papers:
    # Lấy paper đầu tiên để test
    test_paper = papers[paper_to_test]
    paper_id = test_paper['paper_id']
    version = test_paper['version']
    
    print(f"🧪 Testing LatexContentProcessor")
    print(f"📄 Paper: {paper_id} - {version}")
    print(f"📏 Content length: {len(test_paper['content']):,} characters\n")
    
    # BƯỚC 1: Build cây thô (Structure)
    print("="*70)
    print("BƯỚC 1: XÂY DỰNG CÂY CẤU TRÚC THÔ")
    print("="*70)
    builder = LatexStructureBuilder(test_paper['content'], paper_id, version)
    root_tree = builder.build_coarse_tree()
    
    # Đếm số node trước khi xử lý
    def count_nodes(node):
        count = 1
        for child in node.get('children', []):
            count += count_nodes(child)
        return count
    
    nodes_before = count_nodes(root_tree)
    print(f"✅ Số nodes ban đầu (chỉ sections): {nodes_before}\n")
    
    # BƯỚC 2: Process nội dung (Tách câu, hình, toán)
    print("="*70)
    print("BƯỚC 2: XỬ LÝ NỘI DUNG CHI TIẾT")
    print("="*70)
    processor = LatexContentProcessor(paper_id, version)
    processor.process_tree(root_tree)
    
    nodes_after = count_nodes(root_tree)
    print(f"✅ Số nodes sau xử lý: {nodes_after}")
    print(f"📊 Tăng thêm: {nodes_after - nodes_before} nodes (sentences, equations, figures...)\n")
    
    # BƯỚC 3: Thống kê các loại node
    print("="*70)
    print("BƯỚC 3: THỐNG KÊ CÁC LOẠI NODE")
    print("="*70)
    
    node_types = {}
    def count_by_type(node):
        node_type = node['type']
        node_types[node_type] = node_types.get(node_type, 0) + 1
        for child in node.get('children', []):
            count_by_type(child)
    
    count_by_type(root_tree)
    
    # Sắp xếp và hiển thị
    for node_type, count in sorted(node_types.items(), key=lambda x: -x[1]):
        print(f"   {node_type:20s}: {count:4d}")
    
    # BƯỚC 4: In ra một phần cây để kiểm tra
    print("\n" + "="*70)
    print("BƯỚC 4: XEM TRƯỚC CẤU TRÚC CÂY (5 MỨC ĐẦU)")
    print("="*70)
    
    def print_tree_limited(node, indent=0, max_depth=5):
        if indent >= max_depth:
            return
        
        prefix = "  " * indent
        content_preview = ""
        if node.get('raw_content'):
            content_preview = node['raw_content'][:50].replace('\n', ' ')
            content_preview = f" → \"{content_preview}...\""
        
        print(f"{prefix}├─ [{node['type']}] {node['title']}{content_preview}")
        
        # Chỉ in tối đa 3 children đầu tiên để tránh quá dài
        children_to_show = node.get('children', [])[:3]
        for child in children_to_show:
            print_tree_limited(child, indent + 1, max_depth)
        
        if len(node.get('children', [])) > 3:
            print(f"{prefix}  ... ({len(node['children']) - 3} more children```python)")
# Test LatexContentProcessor với paper đầu tiên

if papers:
    # Lấy paper đầu tiên để test
    test_paper = papers[paper_to_test]
    paper_id = test_paper['paper_id']
    version = test_paper['version']
    flattened_content = test_paper['content']
    
    print(f"🧪 Testing LatexContentProcessor")
    print(f"📄 Paper: {paper_id} - {version}")
    print(f"📊 Content length: {len(flattened_content):,} characters\n")
    print("="*70)
    
    # Bước 1: Build cây cấu trúc thô
    print("\n📌 BƯỚC 1: Xây dựng cây cấu trúc thô...")
    builder = LatexStructureBuilder(flattened_content, paper_id, version)
    root_tree = builder.build_coarse_tree()
    
    # Đếm nodes trước khi process
    def count_nodes_by_type(node, counts=None):
        if counts is None:
            counts = {}
        node_type = node['type']
        counts[node_type] = counts.get(node_type, 0) + 1
        for child in node.get('children', []):
            count_nodes_by_type(child, counts)
        return counts
    
    before_counts = count_nodes_by_type(root_tree)
    print(f"✅ Cây thô đã được xây dựng")
    print(f"   Số lượng nodes theo loại:")
    for node_type, count in sorted(before_counts.items()):
        print(f"   - {node_type}: {count}")
    
    # Bước 2: Process content chi tiết
    print(f"\n📌 BƯỚC 2: Xử lý content chi tiết (tách câu, công thức, hình...)...")
    processor = LatexContentProcessor(paper_id, version)
    processor.process_tree(root_tree)
    
    # Đếm nodes sau khi process
    after_counts = count_nodes_by_type(root_tree)
    print(f"✅ Đã xử lý xong!")
    print(f"   Số lượng nodes sau khi xử lý:")
    for node_type, count in sorted(after_counts.items()):
        print(f"   - {node_type}: {count}")
    
    # So sánh sự thay đổi
    print(f"\n📊 THỐNG KÊ THAY ĐỔI:")
    new_types = set(after_counts.keys()) - set(before_counts.keys())
    if new_types:
        print(f"   Các loại node mới được tạo:")
        for node_type in sorted(new_types):
            print(f"   - {node_type}: {after_counts[node_type]} nodes")
    
    # Bước 3: Lưu kết quả
    output_file = f"detailed_structure_{paper_id}_{version}.json"
    print(f"\n📌 BƯỚC 3: Lưu kết quả vào file...")
    builder.print_tree_to_file(root_tree, output_file)
    
    # Hiển thị preview cây chi tiết (chỉ 2 level đầu để khỏi quá dài)
    print(f"\n🌳 PREVIEW CẤU TRÚC CÂY CHI TIẾT (2 levels đầu):")
    print("="*70)
    
    def print_limited_tree(node, indent=0, max_depth=2):
        if indent >= max_depth:
            return
        prefix = "  " * indent
        content_preview = ""
        if node.get('raw_content'):
            preview_text = node['raw_content'][:50].replace('\n', ' ')
            content_preview = f" | Content: {preview_text}..."
        
        print(f"{prefix}├─ [{node['type'].upper()}] {node['title']}{content_preview}")
        
        # Hiển thị số lượng children
        if node['children']:
            print(f"{prefix}   ({len(node['children'])} children)")
            for i, child in enumerate(node['children'][:3]):  # Chỉ show 3 children đầu
                print_limited_tree(child, indent + 1, max_depth)
            if len(node['children']) > 3:
                print(f"{prefix}   ... và {len(node['children']) - 3} children khác")
    
    print_limited_tree(root_tree)
    print("="*70)
    
else:
    print("⚠️ Không có papers để test!")

🧪 Testing LatexContentProcessor
📄 Paper: 2403-00530 - 2403-00530v2
📏 Content length: 51,454 characters

BƯỚC 1: XÂY DỰNG CÂY CẤU TRÚC THÔ
✅ Số nodes ban đầu (chỉ sections): 12

BƯỚC 2: XỬ LÝ NỘI DUNG CHI TIẾT
✅ Số nodes sau xử lý: 249
📊 Tăng thêm: 237 nodes (sentences, equations, figures...)

BƯỚC 3: THỐNG KÊ CÁC LOẠI NODE
   sentence            :  201
   figure              :   20
   equation            :   16
   section             :    8
   subsection          :    3
   document            :    1

BƯỚC 4: XEM TRƯỚC CẤU TRÚC CÂY (5 MỨC ĐẦU)
🧪 Testing LatexContentProcessor
📄 Paper: 2403-00530 - 2403-00530v2
📊 Content length: 51,454 characters


📌 BƯỚC 1: Xây dựng cây cấu trúc thô...
✅ Cây thô đã được xây dựng
   Số lượng nodes theo loại:
   - document: 1
   - section: 8
   - subsection: 3

📌 BƯỚC 2: Xử lý content chi tiết (tách câu, công thức, hình...)...
✅ Đã xử lý xong!
   Số lượng nodes sau khi xử lý:
   - document: 1
   - equation: 16
   - figure: 20
   - section: 8
   - sentence:

detailed_structure_2403-00530_2403-00530v1